morphology.py is a pipeline that calls the statmorph package.  This notebook shows how to use morphology.py in a notebook or interactive setting.

The morphology pipeline requires only one argument at the command line.  The argument is the filename of the stamp you want to analyze.  You will also need a segmentation stamp, i.e., a cutout from the segmentation map that corresponds to the cutout to be analyzed.  The segmentation stamp must use 0 to indicate the absence of source flux in a pixel.  (PyBDSM uses -1 to indicate background in its `island images'.)

Let's use stamp.py to create the stamps and segmentation stamps:

In [9]:
import subprocess
subprocess.call(['../stamps/stamp.py', '--radio_img',
                 '../radio_imgs/img_MACSJ0416_S-BAND_data-MFS-image.fits',
                 '--cat', '../ian_cats/VLA-HFF_0416_compact_optical_rasort.txt', '--imgs',
                 '../radio_imgs/pyrank/img_MACSJ0416_S-BAND_data-MFS-image.pybdsm_pyrank.fits'])

0

The results of the call are stamps from the radio data (\*img0.fits) and stamps from the segmentation image (\*img1.fits).  The pipeline, run from the command line, requires that the segmentation stamp share the root name of the data stamp and end in .seg.fits.  For this example the segmentation stamps must be \*img0.seg.fits.  So let's rename the files:

In [10]:
subprocess.call(['for i in output/*img1.fits; do mv "$i" "${i%%img1.fits}img0.seg.fits"; done'],
                shell = True)

0

Now, import morphology and read in 1 pair of stamps for an example:

In [11]:
from astropy.io import fits
import morphology
img = fits.open('output/113img0.fits')
segm = fits.open('output/113img0.seg.fits')
# pyrank images indicate background with -1; we must indicate background with 0
segm[0].data += 1

morphology.py has 5 main functions: return_bkg, measure_morphology, mod_segmap_1pix, write_gini_mask, and build_table.

The first step is to get the median and RMS of the background in the stamp.  statmorph requires that the stamp has a background of 0.  The segmentation stamp defines the masked region.  The median and RMS are calculated with sigma_clipped_stats.

The inputs to return_bkg are the stamp and the segmentation stamp as a boolean mask.  The outputs are the median of the masked and sigma-clipped region and a 2D array of the same size as the stamp filled with the RMS value.  (We assume the RMS is constant across the stamp.)

In [12]:
bkg_median, bkg_rms_arr = morphology.return_bkg(img[0].data, segm[0].data.astype(bool))

In [13]:
print(bkg_median, bkg_rms_arr)

(1.4006300830260443e-07, array([[  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       ..., 
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07],
       [  9.84536169e-07,   9.84536169e-07,   9.84536169e-07, ...,
          9.84536169e-07,   9.84536169e-07,   9.84536169e-07]]))


The second step is to pass to measure_morphology the stamp, segmentation stamp, median background, background RMS array, and a "cutout extent" parameter, which is passed to statmorph.  You can fiddle with the extent parameter.  I've used 2.

The outputs of measure_morphology are the morphological parameters from statmorph for the object labeled in the central pixel of the segmentation stamp.

In [14]:
obj_morph = morphology.measure_morphology(img[0].data, segm[0].data, bkg_median, bkg_rms_arr, 2)

Finished processing source 550.



In [18]:
print(obj_morph[0].asymmetry)

0.00284578553292


If obj_morph exists, the pipeline will call statmorph an additional 2 times: once with a segmentation stamp whose boundary has been expanded by 1 pixel; and once with a segmentation stamp whose boundary has been shrunk by 1 pixel.  The idea is to estimate the morphological parameters 3 times, under segmentation uncertainty, and then to calculate standard deviations.

mod_segmap_1pix modifies the segmentation stamp.

In [17]:
big_segm, small_segm = morphology.mod_segmap_1pix(segm[0].data)

In [20]:
big_bkg_median, big_bkg_rms_arr = morphology.return_bkg(img[0].data, big_segm.astype(bool))
small_bkg_median, small_bkg_rms_arr = morphology.return_bkg(img[0].data, small_segm.astype(bool))

In [22]:
big_obj_morph = morphology.measure_morphology(img[0].data, big_segm, big_bkg_median, big_bkg_rms_arr, 2)
small_obj_morph = morphology.measure_morphology(img[0].data, small_segm, small_bkg_median, small_bkg_rms_arr, 2)

Finished processing source 1.

Finished processing source 1.



In [26]:
print(big_obj_morph[0].asymmetry)
print(small_obj_morph[0].asymmetry)
obj_morph = [obj_morph[0], big_obj_morph[0], small_obj_morph[0]]

0.00971299061748
0.00157582621604


If both big_obj_morph and small_obj_morph exist, a standard deviation can be written to the output.

The final two functions organize data products.  write_gini_mask outputs the mask used to estimate the Gini and M20 parameters.  The inputs are the stamp data, the stamp header, the measure_morphology object output, and a filename.  build_table extracts the parameters from the obj_morph object.  Its inputs are the measure_morphology object output and the stamp header.

In [25]:
morphology.write_gini_mask(img[0].data, img[0].header, obj_morph[0], 'test.gini.fits')

In [31]:
t = morphology.build_table(obj_morph, img[0].header)
t

asymmetry,RA center for asymmetry,Dec center for asymmetry,mean asym,stddev asym,concentration,mean concen,stddev concen,smoothness,mean smooth,stddev smooth,half light elliptical semimajor axis length,petrosian elliptical semimajor axis length,gini,mean gini,stddev gini,m20,mean m20,stddev m20,flag,S/N per pixel
,deg,deg,,,,,,,,,arcsec,arcsec,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.00284578553292,63.9999783097,-24.0257062098,0.00471153412215,0.00357436469624,2.3052209749,2.30632445889,0.000780609295267,0.0,0.0,0.0,0.541813237613,1.02252307271,0.424375853508,0.428626167208,0.00617456912199,-1.61379944428,-1.61509755854,0.00163644529128,0.0,55.1271029913


The pipeline adds a column with the filename of the stamp.  This is for diagnostic purposes.

In [32]:
t['filename'] = 'output/113img0.fits'
t

asymmetry,RA center for asymmetry,Dec center for asymmetry,mean asym,stddev asym,concentration,mean concen,stddev concen,smoothness,mean smooth,stddev smooth,half light elliptical semimajor axis length,petrosian elliptical semimajor axis length,gini,mean gini,stddev gini,m20,mean m20,stddev m20,flag,S/N per pixel,filename
,deg,deg,,,,,,,,,arcsec,arcsec,,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19
0.00284578553292,63.9999783097,-24.0257062098,0.00471153412215,0.00357436469624,2.3052209749,2.30632445889,0.000780609295267,0.0,0.0,0.0,0.541813237613,1.02252307271,0.424375853508,0.428626167208,0.00617456912199,-1.61379944428,-1.61509755854,0.00163644529128,0.0,55.1271029913,output/113img0.fits


Now we write the table.

In [33]:
t.write('t.cat', format = 'ascii.fixed_width', overwrite = True)

When run in batch mode from the command line, the pipeline writes in the directory with the input stamps a table for each source.  At the end, a concatenated table, allmorphs.cat, is written in the working directory.

Command line usage is simple: the only argument is the stamp filename.

```./morphology.py ../stamps/output/*img0.fits```

NOTE: there is a function in the pipeline called make_segmap.  I wrote this function before acquiring segmentation maps from external sources.  The function does rudimentary source detection, masking, and segmentation map creation.  Use at your own risk.  It may not play well with the pipeline parts that were developed after I obtained segmaps from elsewhere.